In [63]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from sklearn import datasets, linear_model
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
import seaborn as sns

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
            columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [227]:
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])

df.loc[df.a == 1, ['a', 'b']] = [123, 456]

df

,a,b,c
0,123,456,3
1,4,5,6
2,7,8,9


In [256]:
elog = pd.read_csv("../Dataset/HCM02-ELog.csv", index_col=0)
eprofile = pd.read_csv("../Dataset/HCM02-EProfile.csv", index_col=0)
eweather = pd.read_csv("../Dataset/HCM02-EWeather.csv", sep="\t", index_col=0)

In [262]:
#Clean elog dataset

elog["received_time"] = pd.to_datetime(elog["received_time"])

print(missing_values_table(elog))
print(elog.info())
elog.head()


Your selected dataframe has 5 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
<class 'pandas.core.frame.DataFrame'>
Int64Index: 604134 entries, 329060 to 468578
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   customer_id    604134 non-null  object        
 1   meter_id       604134 non-null  object        
 2   received_time  604134 non-null  datetime64[ns]
 3   meter_idx      604134 non-null  float64       
 4   source         604134 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 27.7+ MB
None


,customer_id,meter_id,received_time,meter_idx,source
329060,EX0030895,MEXE6631620,2020-04-01 21:27:01,27530.1600,train
215307,EX0007732,MEXE9037803,2019-12-02 04:33:51,696.4650,train
513451,EX0083465,MEXE8173596,2020-10-05 04:13:01,98926.4745,test
77785,EX0011899,MEXE8290433,2019-05-06 06:07:45,2117.8500,train
428307,EX0003992,MEXE8205919,2020-07-12 20:16:25,5073.6420,train


In [284]:
#Clean eprofile
eprofile["registration_date"] = pd.to_datetime(eprofile["registration_date"])

duplicated_profile = eprofile[eprofile.duplicated(subset=["customer_id"], keep=False)].sort_values(by=["customer_id"], ascending=False)
for id in duplicated_profile.customer_id.unique():
    # Get city and sector
    city = duplicated_profile[(duplicated_profile.customer_id == id) & (duplicated_profile.city.isnull() == False)].iloc[0, 2]
    sector = duplicated_profile[(duplicated_profile.customer_id == id) & (duplicated_profile.sector.isnull() == False)].iloc[0, 3]

    if(pd.isnull(city)  == False and pd.isnull(sector) == False):
        # fillna
        eprofile.loc[eprofile.customer_id == id, ["city", "sector"]] = [city, sector]

eprofile.city.fillna(eprofile.city.mode()[0], inplace=True)
eprofile.sector.fillna(eprofile.sector.mode()[0], inplace=True)

eprofile = eprofile.groupby("customer_id").agg('min').reset_index()

print(missing_values_table(eprofile))
print(eprofile.info())
eprofile.head()

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        159 non-null    object        
 1   registration_date  159 non-null    datetime64[ns]
 2   city               159 non-null    object        
 3   sector             159 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.1+ KB
None


,customer_id,registration_date,city,sector
0,EX0000099,2016-08-21,HNI,Domestic
1,EX0000112,2020-01-09,KTM,Retail
2,EX0000591,2015-04-06,VPC,Entertainment
3,EX0000938,2016-08-01,BDG,Entertainment
4,EX0000990,2020-02-19,SG,Agriculture


In [318]:
eweather["date"] = pd.to_datetime(eweather['date'])

print(missing_values_table(eweather))
print(eweather.info())
eweather.head()

Your selected dataframe has 12 columns.
There are 10 columns that have missing values.
              Missing Values  % of Total Values
pressure               71947               58.8
wind_bearing            9074                7.4
wind_speed              2848                2.3
cloud_cover             1981                1.6
visibility              1439                1.2
temp                    1007                0.8
humidity                 999                0.8
dew_point                998                0.8
trend                    986                0.8
summary                  982                0.8
<class 'pandas.core.frame.DataFrame'>
Int64Index: 122393 entries, 32047 to 101048
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          122393 non-null  datetime64[ns]
 1   city          122393 non-null  object        
 2   temp          121386 non-null  object        
 3   dew_point 

,date,city,temp,dew_point,humidity,cloud_cover,trend,summary,wind_speed,visibility,pressure,wind_bearing
32047,2019-10-31 07:00:00,VPC,21.67,19.25,0.86,0.22,clear-day,Clear,1.41,7.89,NaN,NAN
5626,2019-08-23 10:00:00,DNI,33.93,24.97,0.6,0.75,partly-cloudy-day,Humid and Mostly Cloudy,1.0,10.01,NaN,320.0
13847,2019-03-02 23:00:00,BDG,25.01,22.99,0.89,0.44,partly-cloudy-night,Humid and Partly Cloudy,1.5,5.01,NaN,120.0
75246,2019-02-09 06:00:00,HPG,11.03,11.03,1.0,0.21,fog,Foggy,1.5,1.69,NaN,221.0
101853,2020-11-08 21:00:00,VPC,21.32,18.4,NAN,0.94,cloudy,Overcast,1.69,16.093,1017.1,312.0


In [325]:
mean = eweather.pressure.mean()
count = eweather.shape[0]
fmean_pressure_weather = eweather.pressure.fillna(mean)
mean_pressure_mae = np.prod(np.abs(fmean_pressure_weather - mean)) / count

print(mean_pressure_mae)


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [285]:
eprofilelog = eprofile.merge(elog, on="customer_id")

print(missing_values_table(eprofilelog))
print(eprofilelog.info())
eprofilelog.head()

Your selected dataframe has 8 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []
<class 'pandas.core.frame.DataFrame'>
Int64Index: 514076 entries, 0 to 514075
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   customer_id        514076 non-null  object        
 1   registration_date  514076 non-null  datetime64[ns]
 2   city               514076 non-null  object        
 3   sector             514076 non-null  object        
 4   meter_id           514076 non-null  object        
 5   received_time      514076 non-null  datetime64[ns]
 6   meter_idx          514076 non-null  float64       
 7   source             514076 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 35.3+ MB
None


,customer_id,registration_date,city,sector,meter_id,received_time,meter_idx,source
0,EX0000099,2016-08-21,HNI,Domestic,MEXE7161373,2020-08-18 23:23:48,103851.4680,train
1,EX0000099,2016-08-21,HNI,Domestic,MEXE7161373,2020-04-29 13:30:34,94828.7760,train
2,EX0000099,2016-08-21,HNI,Domestic,MEXE7161373,2019-04-23 20:55:48,65878.8585,train
3,EX0000099,2016-08-21,HNI,Domestic,MEXE7161373,2020-12-28 12:19:02,112814.8560,test
4,EX0000099,2016-08-21,HNI,Domestic,MEXE7161373,2019-08-01 21:38:55,75404.4795,train


In [288]:
eprofilelog[eprofilelog.duplicated(subset=["customer_id", "meter_id", "received_time"], keep=False)].sort_values(by=["customer_id"], ascending=False)

,customer_id,registration_date,city,sector,meter_id,received_time,meter_idx,source


In [316]:
eprofilelog[(eprofilelog.customer_id == "EX0071193")
    & (eprofilelog.meter_id == "MEXE7135465")].groupby([eprofilelog.customer_id, eprofilelog.meter_id, eprofilelog.received_time.dt.date]).agg({"meter_idx": ["max", "min"]}).head()

meter_idx          
                                            max       min
customer_id meter_id    received_time                    
EX0071193   MEXE7135465 2020-11-08     9755.445  9755.340
                        2020-11-09     9755.760  9755.550
                        2020-11-10     9756.075  9755.865
                        2020-11-11     9756.600  9756.180
                        2020-11-12     9756.810  9756.705

In [317]:
eweather.head()

,date,city,temp,dew_point,humidity,cloud_cover,trend,summary,wind_speed,visibility,pressure,wind_bearing
32047,2019/10/31 07:00:00 AM,VPC,21.67,19.25,0.86,0.22,clear-day,Clear,1.41,7.89,NaN,NAN
5626,2019/08/23 10:00:00 AM,DNI,33.93,24.97,0.6,0.75,partly-cloudy-day,Humid and Mostly Cloudy,1.0,10.01,NaN,320.0
13847,2019/03/02 11:00:00 PM,BDG,25.01,22.99,0.89,0.44,partly-cloudy-night,Humid and Partly Cloudy,1.5,5.01,NaN,120.0
75246,2019/02/09 06:00:00 AM,HPG,11.03,11.03,1.0,0.21,fog,Foggy,1.5,1.69,NaN,221.0
101853,2020/11/08 09:00:00 PM,VPC,21.32,18.4,NAN,0.94,cloudy,Overcast,1.69,16.093,1017.1,312.0


In [289]:
print(eprofile.shape)

# eprofilecount = eprofile.groupby("customer_id").customer_id.count()[lambda x: x >= 2]

# print(eprofile[eprofile.city.isnull()])

# print(eprofilecount[eprofilecount.values >= 2])

eprofile[eprofile.duplicated(subset=["customer_id"], keep=False)].sort_values(by=["customer_id"], ascending=False)

# eprofile.city.describe()

(159, 4)


,customer_id,registration_date,city,sector


In [151]:
# elog[(elog.customer_id == "EX0071193")
# & 
# (elog.received_time >= "2015-11-25")
# & 
# ((elog.received_time <= "2020-01-06"))]["meter_id"].value_counts()

# elog[(elog.customer_id == "EX0002468")]["meter_id"].value_counts()

elog[(elog.customer_id == "EX0013504")][lambda x: x.received_time == x.received_time.min()]


,customer_id,meter_id,received_time,meter_idx,source
168202,EX0013504,MEXE9036875,2019-10-04 18:10:05,128.73,train


In [296]:
elog[(elog.customer_id == "EX0071193")]["meter_id"].value_counts()

MEXE7196348    2638
MEXE7135465     216
Name: meter_id, dtype: int64

In [139]:
A = elog.groupby(elog.customer_id).meter_id.nunique()[lambda x: x >= 2].sort_values(ascending=False).index.values

B = eprofile[eprofile.duplicated(subset=["customer_id"], keep=False)].sort_values(by=["customer_id"], ascending=False).customer_id.values

A[np.isin(A, B)]

array(['EX0071193'], dtype=object)